___
# Lets important necessary libraries
___

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import cufflinks as cf
cf.go_offline()

___
# Importing the dataset
___

In [49]:
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')

print("Shape of matches.csv :  " , matches.shape)
print("Shape of deliveres.csv :" , deliveries.shape)

Shape of matches.csv :   (756, 18)
Shape of deliveres.csv : (179078, 21)


In [50]:
matches.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [51]:
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [52]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [53]:
matches.team1.unique()

array(['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

___
# Taking only consistent teams data from complete data
___

In [54]:
# Replacing 'Delhi Daredevils' with 'Delhi Capitals' because its same team
matches.replace(to_replace = [ 'Delhi Daredevils' ,'Rising Pune Supergiant']   , value = ['Delhi Capitals' , 'Rising Pune Supergiants'] , inplace = True)
deliveries.replace(to_replace = ['Delhi Daredevils' , 'Rising Pune Supergiant' ] , value = [ 'Delhi Capitals' , 'Rising Pune Supergiants'], inplace = True)

# consistent_teams = ['Sunrisers Hyderabad', 'Mumbai Indians',
#         'Kolkata Knight Riders', 'Royal Challengers Bangalore', 
#         'Delhi Capitals', 'Kings XI Punjab','Chennai Super Kings', 'Rajasthan Royals']

# matches = matches[ (matches.team1.isin( consistent_teams )) & (matches.team2.isin( consistent_teams ))]
# deliveries = deliveries[ (deliveries.batting_team.isin( consistent_teams )) & (deliveries.bowling_team.isin( consistent_teams )) ]

# Replacing the names of teams to short names of only consistent teams
dic = {'Sunrisers Hyderabad' : 'SRH' , 'Kolkata Knight Riders' : 'KKR',
       'Royal Challengers Bangalore' : 'RCB' , 'Kings XI Punjab' : 'KXIP',
       'Mumbai Indians' : 'MI' , 'Chennai Super Kings' : 'CSK' , 
       'Rajasthan Royals' : 'RR' , 'Delhi Capitals' : 'DC'
      }
matches.replace( dic , inplace = True )
deliveries.replace( dic , inplace = True )

___
# Removing rows with missing values
___

In [55]:
matches.isnull().sum().sort_values(ascending = False)

umpire3            637
city                 7
player_of_match      4
winner               4
umpire1              2
umpire2              2
toss_winner          0
season               0
date                 0
team1                0
team2                0
result               0
toss_decision        0
dl_applied           0
win_by_runs          0
win_by_wickets       0
venue                0
id                   0
dtype: int64

In [56]:
# matches.drop( ['umpire3'] , axis = 1 , inplace = True )

# matches = matches.dropna()
# matches.dropna( axis=0 , how='any',  inplace = True)


# # The disadvantage of above two lines is that now we have breaked index range which would produce Nan values while concatenating in our case
# # Resetting the indexes of rows of train_df
# # remove previous index with new continuous default index 
# matches.reset_index(inplace = True, drop = True)
# matches.shape

In [57]:
deliveries.isnull().sum().sort_values(ascending = False)

fielder             172630
player_dismissed    170244
dismissal_kind      170244
bowler                   0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
wide_runs                0
is_super_over            0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
match_id                 0
dtype: int64

In [58]:
deliveries.drop(['fielder' , 'player_dismissed' , 'dismissal_kind'] , axis = 1 , inplace = True)
deliveries.shape

(179078, 18)

In [59]:
matches.team1.unique()

array(['SRH', 'MI', 'Gujarat Lions', 'Rising Pune Supergiants', 'RCB',
       'KKR', 'DC', 'KXIP', 'CSK', 'RR', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors'], dtype=object)

___
# Let's find out the win %age of each team
___

In [60]:
win_percentage_of_different_teams = ( matches.winner.value_counts() / (  matches.team1.value_counts() + matches.team2.value_counts() ) ) * 100

In [61]:
win_percentage_of_teams = ( matches.winner.value_counts() / (  matches.team1.value_counts() + matches.team2.value_counts() ) ) * 100

In [62]:
win_percentage_of_teams = win_percentage_of_teams.to_frame().reset_index().rename( columns = {'index' : 'Team' , 0 : 'Win %age'})
win_percentage_of_teams.sort_values(by = 'Win %age' , ascending = False , inplace = True)

In [63]:
win_percentage_of_teams.head(15)

,Team,Win %age
0,CSK,60.975610
7,MI,58.288770
12,SRH,53.703704
4,KKR,51.685393
10,RR,51.020408
11,Rising Pune Supergiants,50.000000
9,RCB,46.666667
5,KXIP,46.590909
1,DC,43.502825
3,Gujarat Lions,43.333333


In [64]:
win_percentage_of_teams.iplot(kind = 'bar' , x = 'Team' , y = 'Win %age' , title = 'Win %age of each team[2008 - 2019]' , xTitle = 'Teams' , yTitle = 'Win %age')  

> # *Chennai Super Kings has the best win %age*

___
# Let's find out the team/teams to feature in most number of SEASONS
___

In [65]:
dic = {}
for team in matches.team1.unique():
    dic[team] = 0
    
for value in matches.groupby('season'):
    for team in dic:
        if team in value[1].team1.unique():
            dic[team] += 1
print(dic)

{'SRH': 7, 'MI': 12, 'Gujarat Lions': 2, 'Rising Pune Supergiants': 2, 'RCB': 12, 'KKR': 12, 'DC': 12, 'KXIP': 12, 'CSK': 10, 'RR': 10, 'Deccan Chargers': 5, 'Kochi Tuskers Kerala': 1, 'Pune Warriors': 3}


In [66]:
teams_vs_seasons = pd.DataFrame( dic.items() , columns = ['Team' , 'No. of seasons'])
teams_vs_seasons.sort_values(by = 'No. of seasons' , ascending = False , inplace = True)
teams_vs_seasons.head(15)

,Team,No. of seasons
1,MI,12
4,RCB,12
5,KKR,12
6,DC,12
7,KXIP,12
8,CSK,10
9,RR,10
0,SRH,7
10,Deccan Chargers,5
12,Pune Warriors,3


In [67]:
teams_vs_seasons.iplot(kind = 'bar' , x = 'Team' , y = 'No. of seasons' , title = 'Teams vs No. of seasons in which they appeared[2008 - 2019]' , xTitle = 'Teams' , yTitle = 'No. of seasons') 

> KKR , RCB , KXIP , DC and MI are found to be appeared in all seasons of ipl till 2019

___
# Let's see the percentage of choosing field and bat after winning the toss
___

In [68]:
toss_selection_prcntage = matches.toss_decision.value_counts().to_frame().reset_index().rename( columns = {'index' : 'Choosen' , 'toss_decision' : 'Selection %age'} )
toss_selection_prcntage

,Choosen,Selection %age
0,field,463
1,bat,293


In [69]:
 toss_selection_prcntage['Selection %age'] =  toss_selection_prcntage['Selection %age'] / 743 * 100

In [70]:
toss_selection_prcntage.head()

,Choosen,Selection %age
0,field,62.314939
1,bat,39.434724


In [71]:
import plotly.express as px
px.pie( toss_selection_prcntage , values='Selection %age', names='Choosen', title='Toss Percentage Distribution' ,color_discrete_sequence=px.colors.sequential.RdBu)

___
# Lets find the lucky teams , Teams who have best toss winning percentage
___

In [72]:
toss_win_prcntage = (  matches.toss_winner.value_counts() / ( matches.team1.value_counts() + matches.team2.value_counts() )  ) * 100
toss_win_prcntage = toss_win_prcntage.to_frame().reset_index().rename( columns = {'index' : 'Team' , 0 : 'Toss win %age'} )
toss_win_prcntage.sort_values( by = 'Toss win %age' , ascending = False , inplace = True )

In [73]:
toss_win_prcntage

,Team,Toss win %age
2,Deccan Chargers,57.333333
6,Kochi Tuskers Kerala,57.142857
10,RR,54.421769
0,CSK,54.268293
7,MI,52.406417
4,KKR,51.685393
1,DC,50.847458
3,Gujarat Lions,50.000000
5,KXIP,46.022727
9,RCB,45.000000


In [74]:
toss_win_prcntage.iplot(kind = 'bar' , x = 'Team' , y = 'Toss win %age' , title = 'Teams vs their Toss winning %age[2008 - 2019]' , xTitle = 'Teams' , yTitle = 'Toss winning %age') 

> Chennai Super Kings is found to be the most lucky team because it has a fair toss winning %age with a larger sample size

___
# Let's find out the player who won maximum number of MOM awards
___

In [75]:
man_of_the_match = matches.player_of_match.value_counts().to_frame().reset_index().rename( columns = {'index' : 'Player' , 'player_of_match' : 'No. of MOM'} )
man_of_the_match.head()

,Player,No. of MOM
0,CH Gayle,21
1,AB de Villiers,20
2,DA Warner,17
3,RG Sharma,17
4,MS Dhoni,17


In [76]:
man_of_the_match.iplot(kind = 'scatter' , x = 'Player' , y = 'No. of MOM' , title = 'Players vs No. of MOM award won by them[2008 - 2019]' , xTitle = 'Players' , yTitle = "No. of MOM's")

___
# Let's find out the percentage of choosing bat and field , given that that particular team had won the toss
___

In [77]:
lis = []
for value in matches.groupby('toss_winner').toss_decision:
    bat_opted   = value[1].value_counts().bat
    field_opted = value[1].value_counts().field
    total       = value[1].value_counts().bat + value[1].value_counts().field
    
    bat_select_prcntage   = ( bat_opted   / total ) * 100
    field_select_prcntage = ( field_opted / total ) * 100
    lis.append( [value[0] , bat_select_prcntage , field_select_prcntage ] )
    

chances_of_choosing = pd.DataFrame( lis , columns = ['Team' , 'Bat Select %age' , 'Field Select %age'])

In [78]:
chances_of_choosing

,Team,Bat Select %age,Field Select %age
0,CSK,53.932584,46.067416
1,DC,34.444444,65.555556
2,Deccan Chargers,55.813953,44.186047
3,Gujarat Lions,6.666667,93.333333
4,KKR,32.608696,67.391304
5,KXIP,32.098765,67.901235
6,Kochi Tuskers Kerala,37.500000,62.500000
7,MI,44.897959,55.102041
8,Pune Warriors,55.000000,45.000000
9,RCB,24.691358,75.308642


In [79]:
fig = go.Figure(data=[
    go.Bar(name="Bat selection %age",  x = chances_of_choosing.Team , y = chances_of_choosing['Bat Select %age']),
    go.Bar(name='Bowl selection %age', x = chances_of_choosing.Team , y = chances_of_choosing['Field Select %age'])
    

])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()


___
# Percentage of matches played at different stadiums
___

In [80]:
venues_data = dict(matches.venue.value_counts())

# Converting dict to DataFrame
venues_data = pd.DataFrame( venues_data.items() ).rename( columns = {0 : 'Stadium' , 1 : '%age of Matches hosted'})
venues_data.head(10)

,Stadium,%age of Matches hosted
0,Eden Gardens,77
1,Wankhede Stadium,73
2,M Chinnaswamy Stadium,73
3,Feroz Shah Kotla,67
4,"Rajiv Gandhi International Stadium, Uppal",56
5,"MA Chidambaram Stadium, Chepauk",49
6,Sawai Mansingh Stadium,47
7,"Punjab Cricket Association Stadium, Mohali",35
8,Maharashtra Cricket Association Stadium,21
9,Subrata Roy Sahara Stadium,17


In [81]:
# Pie Chart
fig = px.pie( venues_data , values='%age of Matches hosted', names='Stadium', title = 'Stadium vs %age of matches hosted comparison [2008 - 2019]'  )
fig.show()

___
# General win %age of each team
___

In [82]:
teams_win_prcntage = (matches.winner.value_counts() /  (matches.team1.value_counts() + matches.team2.value_counts()) ) * 100


# Converting series to DataFrame
teams_win_prcntage = pd.DataFrame(teams_win_prcntage).reset_index().rename( columns = {'index' : 'Teams' , 0 : 'Win %age'})
teams_win_prcntage.sort_values( by = 'Win %age' , ascending = False , inplace = True)
teams_win_prcntage.head()

,Teams,Win %age
0,CSK,60.975610
7,MI,58.288770
12,SRH,53.703704
4,KKR,51.685393
10,RR,51.020408


In [83]:
teams_win_prcntage.iplot(kind = 'bar' , x = 'Teams' , y = 'Win %age' , title = 'Teams vs Win %age' , xTitle = 'Teams' , yTitle = "Win %age")

> CSK has best win %age

___
# Importance of Toss : Let's find out the winning percentage of each team after winning the TOSS
___

In [84]:
df = matches[ matches.toss_winner ==  matches.winner ]

win_prcntage_after_winning_toss = (df.winner.value_counts() / matches.toss_winner.value_counts()) * 100

# converting series to DataFrame
win_prcntage_after_winning_toss = pd.DataFrame( win_prcntage_after_winning_toss ).reset_index().rename(columns = {'index' : 'Teams' , 0 : 'Win %age after winning TOSS'})

# Sorting the values of the dataframe
win_prcntage_after_winning_toss.sort_values(by = 'Win %age after winning TOSS' , ascending = False , inplace = True)
win_prcntage_after_winning_toss.head()

,Teams,Win %age after winning TOSS
3,Gujarat Lions,66.666667
0,CSK,64.044944
11,Rising Pune Supergiants,61.538462
4,KKR,57.608696
7,MI,57.142857


In [85]:
win_prcntage_after_winning_toss.iplot(kind = 'bar' , x = 'Teams' , y = 'Win %age after winning TOSS' , title = 'Teams vs Win %age after winning TOSS' , xTitle = 'Teams' , yTitle = "Win %age after winning TOSS")

> I would rate CSK to be the team with a fair Win %age after winning TOSS due to its large sample size

___
# Relation of Toss decision with Season Number
___

In [86]:
dic = dict(matches.groupby('season').toss_decision.value_counts())
lis = []
for season in matches.season.unique():
    f = dic[(season , 'field')]
    b = dic[(season , 'bat')]
    t = f + b
    field_select_prcntage = ( f / (f + b) ) * 100 
    bat_select_prcntage = ( b / (f + b) ) * 100
    lis.append( [season , field_select_prcntage , bat_select_prcntage  ] )

# Converting list to DataFrame
season_vs_decision = pd.DataFrame( lis ).rename(columns = {0 : 'Season' , 1 : 'Field_select_%age' , 2 : 'Bat_select_%age'})
season_vs_decision.head()


,Season,Field_select_%age,Bat_select_%age
0,2017,81.355932,18.644068
1,2008,55.172414,44.827586
2,2009,38.596491,61.403509
3,2010,35.000000,65.000000
4,2011,65.753425,34.246575


In [87]:
fig = go.Figure(data=[
    go.Bar(name="Field selection %age",  x = season_vs_decision.Season  , y = season_vs_decision['Field_select_%age']),
    go.Bar(name='Bat selection %age', x = season_vs_decision.Season , y = season_vs_decision['Bat_select_%age'])
    

])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

> From the above we find that Teams now likes to chase rather than to give target first

___
# Let's add season number to matches and form a new df named as updated_df
___


In [88]:
# Adding season number to matches 
updated_df = pd.merge( matches[ ['id' , 'season'] ] , deliveries , left_on = 'id' , right_on = 'match_id' )

In [89]:
updated_df.head()

,id,season,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs
0,1,2017,1,1,SRH,RCB,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0
1,1,2017,1,1,SRH,RCB,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0
2,1,2017,1,1,SRH,RCB,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4
3,1,2017,1,1,SRH,RCB,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0
4,1,2017,1,1,SRH,RCB,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2


In [90]:
updated_df.total_runs.sum()

235290

___
# Total runs scored across various seasons
___

In [91]:
df = updated_df.groupby('season').total_runs.sum()
df = df.reset_index()
df.head(12)

,season,total_runs
0,2008,17937
1,2009,16353
2,2010,18883
3,2011,21154
4,2012,22453
5,2013,22602
6,2014,18931
7,2015,18353
8,2016,18862
9,2017,18786


In [92]:
df.iplot(kind = 'bar' , x = 'season' , y = 'total_runs' , title = 'Season vs Total Runs Scored' , xTitle = 'Seasons' , yTitle = "Total Runs Scored")

In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df['season'] , y = df['total_runs'],
                    mode='lines+markers',
                    name='lines'))

___
# Let's find out the number of fours and sixes across different seasons
___

In [94]:
matches_2.head()

NameError: name 'matches_2' is not defined